In [1]:
#!sudo apt-get install python3-opencv

!pip3 install mtcnn
!pip3 install git+https://github.com/rcmalli/keras-vggface.git

    100% |################################| 2.3MB 269kB/s ta 0:00:01
  Running setup.py bdist_wheel for mtcnn ... done
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-m_kdphuj-build
  Running setup.py install for keras-vggface ... done


In [27]:
#!pip3 install boto3

In [1]:
import cv2 as cv
from PIL import Image
import sys
import os
import urllib
import tensorflow.contrib.tensorrt as trt
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as pyplot
import matplotlib.patches as patches
import tensorflow as tf
import numpy as np
import time
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from numpy import asarray

%matplotlib inline

Using TensorFlow backend.


In [2]:
import send_email_attach_aws_ses as sendemail

In [3]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
        # load image from file
        pixels = pyplot.imread(filename)
        # create the detector, using default weights
        detector = MTCNN()
        # detect faces in the image
        results = detector.detect_faces(pixels)
        # extract the bounding box from the first face
        x1, y1, width, height = results[0]['box']
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        return face_array


# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
        # extract faces
        faces = [extract_face(f) for f in filenames]
        # convert into an array of samples
        samples = asarray(faces, 'float32')
        # prepare the face for the model, e.g. center pixels
        samples = preprocess_input(samples, version=2)
        # create a vggface model
        model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
        # perform prediction
        yhat = model.predict(samples)
        return yhat

# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
        # calculate distance between embeddings
        score = cosine(known_embedding, candidate_embedding)
        matched = score <= thresh
        
        return matched, score, thresh

In [4]:
rishi_img = ['images/rishi.jpeg']
rishi_240_img = ['images/rishi-240.jpg']
sharon_stone1_img = ['images/sharon_stone1.jpg']

In [5]:
rm_embed = get_embeddings(rishi_img)
rm_240_embed = get_embeddings(rishi_240_img)
ss_embed = get_embeddings(sharon_stone1_img)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [6]:
def print_match(isMatched):
    if isMatched[0]:
        print('>face is a Match (%.3f <= %.3f)' % (isMatched[1], isMatched[2]))
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (isMatched[1], isMatched[2]))

In [7]:
print("Positive Test")
print_match(is_match(rm_embed, rm_240_embed))
print_match(is_match(rm_embed, rm_embed))

print("Negative Test")
print_match(is_match(rm_embed, ss_embed))

Positive Test
>face is a Match (0.077 <= 0.500)
>face is a Match (-0.000 <= 0.500)
Negative Test
>face is NOT a Match (0.801 > 0.500)


In [8]:
def get_resized_img(image):
    #plt.imshow(image)
    return np.array(image.resize((20, 20)))

In [9]:
def get_wm_image(cp, index):          
    ret, frame = cp.read()
    impath = "images/face" + str(index) + ".jpg"    
    
    #image_resized = get_resized_img(frame) 
    
    cv.imwrite(impath, frame)    
    return impath

In [10]:
def capture_camp_and_detect(num_of_samples):
    cap = cv.VideoCapture(1)
    try:
    
        count = 0

        while(count < num_of_samples):
            cur_img_path = get_wm_image(cap, count)
            
            print(cur_img_path)               
                                  
            unknown_embed = get_embeddings([cur_img_path])
            isMatched = is_match(rm_embed, unknown_embed)
            
            print_match(isMatched)
                        
            if isMatched[0] == False:
                sendemail.send_email(cur_img_path, "Security Alert - Door 2", "Unidentified Person")
            
            os.remove(cur_img_path)
            
            count += 1

    finally:
        cap.release()
        cv.destroyAllWindows()

In [11]:
t0 = time.time()

capture_camp_and_detect(1)

t1 = time.time()
print('Runtime: %f seconds' % (float(t1 - t0)))

images/face0.jpg
>face is a Match (0.440 <= 0.500)
Email sent! Message ID:
0100016c3964803d-82061fc5-818e-4639-b33e-2af9ed4b8d7a-000000
Runtime: 62.817552 seconds
